## Installing Required Modules

In [2]:
pip install opencv-python-headless

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install torch torchvision

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install flask

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
!pip install pyHS100

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


## Importing Libraries

In [6]:
from flask import Flask, Response  # Import Flask framework for building web applications
from pyHS100 import SmartPlug  # Import SmartPlug class for controlling TP-Link Smart Plugs
import cv2  # Import OpenCV library for image and video processing
import torch  # Import PyTorch library for deep learning
import time  # Import time module for timing operations

## Testing Smart plug connection and control

In [7]:
# IP address of our smart plug
plug_ip = '10.0.0.125'

# Establishing connection to our smart plug
plug = SmartPlug(plug_ip)


##### Turning smart plug on

In [8]:
plug.turn_on()

##### Turning smart plug off

In [9]:
plug.turn_off()

## Object Detection with Smart Plug Control
real-time object detection system using the YOLOv5 model. It captures video frames from a webcam, performs object detection to count the number of people in the frame, and displays the count on the video feed. Additionally, it controls a smart plug based on the number of people detected: turning the plug on when people are detected and off otherwise.

##### Initialize Flask and connect to smartplug

In [10]:
app = Flask(__name__)

plug_ip = '10.0.0.125'
plug = SmartPlug(plug_ip)

##### Load YOLOv5 model for object detection

In [11]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)

Using cache found in /home/jetson/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-5-8 Python-3.8.10 torch-1.12.0a0+2c916ef.nv22.3 CUDA:0 (Orin, 7337MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


##### Check if GPU is available, otherwise use CPU

In [12]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)
print("Using device:", device)

Using device: cuda


##### Function to generate video frames with object detection

In [13]:
def generate_frames():
    # Open the default camera for video capture
    cap = cv2.VideoCapture(0)

    while True:
        # Read a frame from the camera
        ret, frame = cap.read()
        if not ret:
            break

        start_time = time.time()  # Start time for overall frame processing

        # Convert frame to RGB color space
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Perform object detection using the YOLOv5 model
        gpu_start_time = time.time()  # Start time for inference
        results = model(frame_rgb)
        gpu_end_time = time.time()  # End time for inference
        gpu_processing_time = gpu_end_time - gpu_start_time  # processing time

        n_people = 0  # Initialize number of people detected
        for det in results.xyxy[0]:
            if int(det[5]) == 0:  # Check if the detected class is 'person'
                n_people += 1  # Increment number of people detected
                start_point = (int(det[0]), int(det[1]))  
                end_point = (int(det[2]), int(det[3]))  
                color = (255, 0, 0)  
                thickness = 2
                cv2.rectangle(frame, start_point, end_point, color, thickness) 

        # Add text to the frame showing number of people detected and processing time
        cv2.putText(frame, f'People Count: {n_people}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(frame, f'{device} Time: {gpu_processing_time:.3f} sec', (50, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2, cv2.LINE_AA)

        # Control smart plug based on number of people detected
        if n_people > 0:
            plug.turn_on()  
        else:
            plug.turn_off()  

        end_time = time.time()  # End time for overall frame processing
        processing_time = end_time - start_time  # Total processing time

        # Encode frame as JPEG
        ret, buffer = cv2.imencode('.jpg', frame)
        frame = buffer.tobytes()

        # Yield the frame for streaming
        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')


##### Route to stream the video feed

In [14]:
@app.route('/video')
def video():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

##### Main to run flask

In [ ]:
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://10.0.0.66:5000
Press CTRL+C to quit
10.0.0.247 - - [08/May/2024 14:08:51] "GET /video HTTP/1.1" 200 -
10.0.0.247 - - [08/May/2024 14:10:15] "GET /video HTTP/1.1" 200 -


## Test with CPU

In [ ]:
from flask import Flask, Response
from pyHS100 import SmartPlug
import cv2
import torch
import time

app = Flask(__name__)

plug_ip = '10.0.0.125'
plug = SmartPlug(plug_ip)

model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True, force_reload=True)
device = 'cpu'
model.to(device)
print("Using device:", device)

def generate_frames():
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        start_time = time.time()

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        gpu_start_time = time.time()
        results = model(frame_rgb)
        gpu_end_time = time.time()
        gpu_processing_time = gpu_end_time - gpu_start_time

        n_people = 0
        for det in results.xyxy[0]:
            if int(det[5]) == 0:
                n_people += 1
                start_point = (int(det[0]), int(det[1]))
                end_point = (int(det[2]), int(det[3]))
                color = (255, 0, 0)
                thickness = 2
                cv2.rectangle(frame, start_point, end_point, color, thickness)

        cv2.putText(frame, f'People Count: {n_people}', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(frame, f'CPU Processing Time: {gpu_processing_time:.3f} sec', (50, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2, cv2.LINE_AA)

        if n_people > 0:
            plug.turn_on()
        else:
            plug.turn_off()

        end_time = time.time()
        processing_time = end_time - start_time

        ret, buffer = cv2.imencode('.jpg', frame)
        frame = buffer.tobytes()

        yield (b'--frame\r\n'
               b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

@app.route('/video')
def video():
    return Response(generate_frames(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)
